In [48]:
from dataset import SpotifyPlaylistDataset
import torch
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import numpy as np
from data_utils import print_playlist_track_counts

In [49]:
test_data = '../_data/main_set.json'

In [50]:
print_playlist_track_counts(test_data)

Playlist Track Counts
--------------------
Vantage ⏣ 🜔 ⌬: 350 tracks
Awake ⏣ ⌬: 238 tracks
Dissolve ⏣ ⌬: 130 tracks
Aeon 2 ⏣ ⌬: 120 tracks
Horizon ⏣ ⌬: 126 tracks
Good Life ⏣ ⌬: 124 tracks
WALK UP LIKE 🜂: 308 tracks
Evolution 🜄: 199 tracks
Simp DM 🜄: 152 tracks
Glitch Mind 🜂🜁: 272 tracks
Overclock 🜁: 109 tracks
Critical Path 🜃: 508 tracks
CRANK 🜍🜂: 112 tracks
Bump Beats 🜍: 249 tracks
Keep it smooth 🜩: 111 tracks
🜊 Hoppin: 203 tracks
Ain't Noise Pollution: 363 tracks
🝓☉ Alt n shit ☉🝓: 151 tracks
🜻 Chef Up: 118 tracks
Reggae: 127 tracks
☉ Pacific: 397 tracks
Indie: 308 tracks
Creekwalk: 310 tracks
Chill/Solo Guitar: 207 tracks
☿gone beyond going☿: 104 tracks
--------------------
Total playlists: 25


In [51]:
# Load the dataset
dataset = SpotifyPlaylistDataset(test_data)

In [52]:
# Get the total number of samples
n_samples = len(dataset)

# Create indices for all samples
indices = list(range(n_samples))

# Get unique playlist IDs
playlist_ids = dataset.playlist_ids

# Create the split
train_indices, test_indices = train_test_split(
    indices, 
    test_size=0.2,  # 20% for testing, 80% for training
    stratify=playlist_ids,  # This ensures that the split maintains the proportion of samples for each playlist
    random_state=42  # For reproducibility
)

# Create Subset objects
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

print(f"Total samples: {n_samples}")
print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")

Total samples: 5396
Training samples: 4316
Testing samples: 1080


In [53]:
batch_size = 32  # You can adjust this based on your needs and computational resources

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Centroids

In [55]:
def compute_playlist_centroids(dataset, indices):
    playlist_features = defaultdict(list)
    for idx in indices:
        playlist_name = dataset.playlist_names[idx]
        features = dataset.features[idx]
        playlist_features[playlist_name].append(features)
    
    playlist_centroids = {}
    for playlist, features in playlist_features.items():
        playlist_centroids[playlist] = np.mean(features, axis=0)
    
    return playlist_centroids

train_playlist_centroids = compute_playlist_centroids(dataset, train_indices)

In [56]:
def classify_song(song_features, playlist_centroids, threshold=0.9):
    similarities = {}
    for playlist, centroid in playlist_centroids.items():
        similarity = cosine_similarity([song_features], [centroid])[0][0]
        similarities[playlist] = similarity
    
    # Sort playlists by similarity
    sorted_playlists = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    
    # Select playlists above the threshold
    selected_playlists = []
    max_similarity = sorted_playlists[0][1]
    for playlist, similarity in sorted_playlists:
        if similarity >= max_similarity * threshold:
            selected_playlists.append(playlist)
        else:
            break
    
    return selected_playlists

In [57]:
from sklearn.metrics import precision_recall_fscore_support

def evaluate_multi_label(dataset, indices, playlist_centroids, threshold=0.9):
    true_labels = []
    predicted_labels = []
    
    for idx in indices:
        song_features = dataset.features[idx]
        true_playlist = dataset.playlist_names[idx]
        predicted_playlists = classify_song(song_features, playlist_centroids, threshold)
        
        # Create binary vectors for true and predicted labels
        all_playlists = list(playlist_centroids.keys())
        true_vector = [1 if playlist == true_playlist else 0 for playlist in all_playlists]
        pred_vector = [1 if playlist in predicted_playlists else 0 for playlist in all_playlists]
        
        true_labels.append(true_vector)
        predicted_labels.append(pred_vector)
    
    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
    
    return precision, recall, f1

# Evaluate
precision, recall, f1 = evaluate_multi_label(dataset, test_indices, train_playlist_centroids)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.2879
Recall: 0.4074
F1 Score: 0.3044


In [58]:
def print_playlist_metrics(dataset, indices, playlist_centroids, threshold=0.9):
    true_labels = []
    predicted_labels = []
    all_playlists = list(playlist_centroids.keys())
    
    for idx in indices:
        song_features = dataset.features[idx]
        true_playlist = dataset.playlist_names[idx]
        predicted_playlists = classify_song(song_features, playlist_centroids, threshold)
        
        true_vector = [1 if playlist == true_playlist else 0 for playlist in all_playlists]
        pred_vector = [1 if playlist in predicted_playlists else 0 for playlist in all_playlists]
        
        true_labels.append(true_vector)
        predicted_labels.append(pred_vector)
    
    precision, recall, f1, support = precision_recall_fscore_support(true_labels, predicted_labels, average=None)
    
    print("\nPlaylist Metrics:")
    print("Playlist\t\tPrecision\tRecall\t\tF1 Score\tSupport")
    for i, playlist in enumerate(all_playlists):
        print(f"{playlist:<20}\t{precision[i]:.4f}\t\t{recall[i]:.4f}\t\t{f1[i]:.4f}\t\t{support[i]}")

# Print detailed metrics
print_playlist_metrics(dataset, test_indices, train_playlist_centroids)


Playlist Metrics:
Playlist		Precision	Recall		F1 Score	Support
Awake ⏣ ⌬           	0.3051		0.3750		0.3364		48
Keep it smooth 🜩    	0.1273		0.3182		0.1818		22
Bump Beats 🜍        	0.3563		0.6200		0.4526		50
Horizon ⏣ ⌬         	0.0933		0.2800		0.1400		25
WALK UP LIKE 🜂      	0.3402		0.5323		0.4151		62
🝓☉ Alt n shit ☉🝓    	0.1233		0.3000		0.1748		30
Chill/Solo Guitar   	0.4405		0.9024		0.5920		41
Creekwalk           	0.3478		0.5161		0.4156		62
Critical Path 🜃     	0.5172		0.1471		0.2290		102
Ain't Noise Pollution	0.4231		0.3014		0.3520		73
Evolution 🜄         	0.0964		0.2000		0.1301		40
Vantage ⏣ 🜔 ⌬       	0.3861		0.5571		0.4561		70
Glitch Mind 🜂🜁      	0.1579		0.2778		0.2013		54
🜻 Chef Up           	0.1235		0.4167		0.1905		24
🜊 Hoppin            	0.2525		0.6098		0.3571		41
Dissolve ⏣ ⌬        	0.1047		0.3462		0.1607		26
Indie               	0.2500		0.1935		0.2182		62
☉ Pacific           	0.3288		0.3038		0.3158		79
Reggae              	0.2250		0.7200		0.3429		25
Aeon 2 ⏣ ⌬          	0

## Neural Net

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from dataset import SpotifyPlaylistDataset
from dataset import SpotifyPlaylistDataset
from torch.utils.data import Subset, DataLoader



# Assuming we still have our dataset and splits from before
dataset = SpotifyPlaylistDataset('../_data/main_set.json')
# ... (train_indices and test_indices creation code here)


# Get the total number of samples
n_samples = len(dataset)

# Create indices for all samples
indices = list(range(n_samples))

# Get unique playlist IDs
playlist_ids = dataset.playlist_ids

# Create the split
train_indices, test_indices = train_test_split(
    indices, 
    test_size=0.2,  # 20% for testing, 80% for training
    stratify=playlist_ids,  # This ensures that the split maintains the proportion of samples for each playlist
    random_state=42  # For reproducibility
)

# Create Subset objects
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

print(f"Total samples: {n_samples}")
print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")

# Get all unique playlist names
all_playlists = list(set(dataset.playlist_names))

# Create multi-label binarizer
mlb = MultiLabelBinarizer()
mlb.fit([all_playlists])  # Fit with all possible playlist names

# Prepare data for PyTorch
X_train = torch.FloatTensor(dataset.features[train_indices])
y_train = torch.FloatTensor(mlb.transform([[playlist] for playlist in np.array(dataset.playlist_names)[train_indices]]))
X_test = torch.FloatTensor(dataset.features[test_indices])
y_test = torch.FloatTensor(mlb.transform([[playlist] for playlist in np.array(dataset.playlist_names)[test_indices]]))

# Create DataLoader objects
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

Total samples: 5396
Training samples: 4316
Testing samples: 1080


In [31]:
class PlaylistClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(PlaylistClassifier, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.layer3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.layer3(x))
        return x

# Initialize the model
input_size = X_train.shape[1]
hidden_size = 64
num_classes = len(all_playlists)
model = PlaylistClassifier(input_size, hidden_size, num_classes)

# Define loss function and optimizer
criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters())
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

In [44]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

def evaluate_model(model, test_loader, threshold=0.5):
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predictions = (outputs >= threshold).float()
            all_predictions.extend(predictions.numpy())
            all_labels.extend(labels.numpy())
    
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
    
    return precision, recall, f1

In [45]:
# Train the model
train_model(model, train_loader, criterion, optimizer)

# Evaluate the model
precision, recall, f1 = evaluate_model(model, test_loader)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Epoch [1/10], Loss: 0.1257
Epoch [2/10], Loss: 0.1148
Epoch [3/10], Loss: 0.1088
Epoch [4/10], Loss: 0.1077
Epoch [5/10], Loss: 0.0955
Epoch [6/10], Loss: 0.0888
Epoch [7/10], Loss: 0.1010
Epoch [8/10], Loss: 0.1006
Epoch [9/10], Loss: 0.0938
Epoch [10/10], Loss: 0.0996
Precision: 0.3867
Recall: 0.1907
F1 Score: 0.2438


/Users/c2k/Projects/spotify-playlist-ai/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [46]:
def print_playlist_metrics(model, test_loader, mlb, threshold=0.5):
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predictions = (outputs >= threshold).float()
            all_predictions.extend(predictions.numpy())
            all_labels.extend(labels.numpy())
    
    precision, recall, f1, support = precision_recall_fscore_support(all_labels, all_predictions, average=None)
    
    print("\nPlaylist Metrics:")
    print("Playlist\t\tPrecision\tRecall\t\tF1 Score\tSupport")
    for i, playlist in enumerate(mlb.classes_):
        print(f"{playlist:<20}\t{precision[i]:.4f}\t\t{recall[i]:.4f}\t\t{f1[i]:.4f}\t\t{support[i]}")

# Print detailed metrics
print_playlist_metrics(model, test_loader, mlb)


Playlist Metrics:
Playlist		Precision	Recall		F1 Score	Support
Aeon 2 ⏣ ⌬          	0.0000		0.0000		0.0000		24
Ain't Noise Pollution	0.7826		0.2466		0.3750		73
Awake ⏣ ⌬           	0.0000		0.0000		0.0000		48
Bump Beats 🜍        	0.6471		0.4400		0.5238		50
CRANK 🜍🜂            	0.0000		0.0000		0.0000		22
Chill/Solo Guitar   	0.9032		0.6829		0.7778		41
Creekwalk           	0.7143		0.3226		0.4444		62
Critical Path 🜃     	0.7241		0.4118		0.5250		102
Dissolve ⏣ ⌬        	0.0000		0.0000		0.0000		26
Evolution 🜄         	0.0000		0.0000		0.0000		40
Glitch Mind 🜂🜁      	0.0000		0.0000		0.0000		54
Good Life ⏣ ⌬       	0.0000		0.0000		0.0000		25
Horizon ⏣ ⌬         	0.0000		0.0000		0.0000		25
Indie               	0.0000		0.0000		0.0000		62
Keep it smooth 🜩    	0.0000		0.0000		0.0000		22
Overclock 🜁         	0.6667		0.0909		0.1600		22
Reggae              	0.3333		0.0400		0.0714		25
Simp DM 🜄           	0.0000		0.0000		0.0000		30
Vantage ⏣ 🜔 ⌬       	0.8261		0.5429		0.6552		70
WALK UP LIKE 🜂      	0

/Users/c2k/Projects/spotify-playlist-ai/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
